In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import rpy2.rinterface_lib.callbacks
import logging
import scrublet as scr
from rpy2.robjects import pandas2ri
import anndata2ri

In [2]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_header()

scanpy==1.7.2 anndata==0.7.6 umap==0.5.1 numpy==1.19.5 scipy==1.6.3 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 louvain==0.7.0


In [3]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths("/home/spuccio/anaconda3/envs/singlecell/lib/R/library/")

library(clustree)


In [4]:
path_CRC="/home/spuccio/isilon/spuccio/SP025_NaClTcell/PangenomeBlueprint/CRC_counts/"
adata_CRC = sc.read("".join([path_CRC,"matrix.mtx"]), cache=True)
adata_CRC = adata_CRC.transpose()
adata_CRC.X = adata_CRC.X.toarray()
barcodes = pd.read_csv("".join([path_CRC,"barcodes.tsv"]), header=None, sep='\t')
genes = pd.read_csv("".join([path_CRC,"genes.tsv"]), header=None, sep='\t')
#Annotate data
barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata_CRC.obs = barcodes
genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata_CRC.var = genes
Metadata_CRC = pd.read_csv("/home/spuccio/isilon/spuccio/SP025_NaClTcell/PangenomeBlueprint/CRC_metadata_2.csv",header=0,low_memory=False)
adata_CRC.obs['CellId'] = Metadata_CRC["Cell"].to_list()
adata_CRC.obs['CellFromTumor'] = Metadata_CRC["CellFromTumor"].to_list()
adata_CRC.obs['PatientNumber'] = Metadata_CRC["PTZ_PatientNumber"].to_list()
adata_CRC.obs['TumorType'] = Metadata_CRC["TumorType"].to_list()
adata_CRC.obs['TumorSite'] = Metadata_CRC["TumorSite"].to_list()
adata_CRC.obs['CellType'] = Metadata_CRC["CellType"].to_list()

... reading from cache file cache/home-spuccio-isilon-spuccio-SP025_NaClTcell-PangenomeBlueprint-CRC_counts-matrix.h5ad


In [5]:
adata_CRC_Tcell = adata_CRC[adata_CRC.obs['CellType']  == "T_cell",:]
adata_CRC_Cancer = adata_CRC[adata_CRC.obs['CellType']  == "Cancer",:]

In [6]:
adata_CRC_Tcell

View of AnnData object with n_obs × n_vars = 8788 × 33694
    obs: 'CellId', 'CellFromTumor', 'PatientNumber', 'TumorType', 'TumorSite', 'CellType'
    var: 'gene_id'

In [7]:
# mitochondrial genes
adata_CRC_Tcell.var['mt'] = adata_CRC_Tcell.var_names.str.startswith('MT-') 
# ribosomal genes
adata_CRC_Tcell.var['ribo'] = adata_CRC_Tcell.var_names.str.startswith(("RPS","RPL"))
# hemoglobin genes.
adata_CRC_Tcell.var['hb'] = adata_CRC_Tcell.var_names.str.contains(("^HB[^(P)]"))

adata_CRC_Tcell.var

Trying to set attribute `.var` of view, copying.


,gene_id,mt,ribo,hb
gene_symbol,,,,
RP11-34P13.3,RP11-34P13.3,False,False,False
FAM138A,FAM138A,False,False,False
OR4F5,OR4F5,False,False,False
RP11-34P13.7,RP11-34P13.7,False,False,False
RP11-34P13.8,RP11-34P13.8,False,False,False
...,...,...,...,...
AC233755.2,AC233755.2,False,False,False
AC233755.1,AC233755.1,False,False,False
AC240274.1,AC240274.1,False,False,False


In [8]:
adata_CRC_Tcell.var['mt'].value_counts()

False    33681
True        13
Name: mt, dtype: int64

In [9]:
adata_CRC_Tcell.var['ribo'].value_counts()

False    33591
True       103
Name: ribo, dtype: int64

In [10]:
adata_CRC_Tcell.var['hb'].value_counts()

False    33682
True        12
Name: hb, dtype: int64

In [11]:
sc.pp.calculate_qc_metrics(adata_CRC_Tcell, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)

In [12]:
print(adata_CRC_Tcell.n_obs, adata_CRC_Tcell.n_vars)
malat1 = adata_CRC_Tcell.var_names.str.startswith('MALAT1')
# we need to redefine the mito_genes since they were first 
# calculated on the full object before removing low expressed genes.
mito_genes = adata_CRC_Tcell.var_names.str.startswith('MT-')
hb_genes = adata_CRC_Tcell.var_names.str.contains('^HB[^(P)]')
ribo_genes = adata_CRC_Tcell.var_names.str.startswith(("RPS","RPL"))

remove = np.add(mito_genes, malat1,ribo_genes)
remove = np.add(remove, hb_genes)
keep = np.invert(remove)

adata_CRC_Tcell = adata_CRC_Tcell[:,keep]

print(adata_CRC_Tcell.n_obs, adata_CRC_Tcell.n_vars)

8788 33694
8788 33668


In [13]:
adata_CRC_Tcell.raw = adata_CRC_Tcell

In [14]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["external_gene_name", "start_position", "end_position", "chromosome_name","gene_biotype"],).set_index("external_gene_name")

In [15]:
annot = pd.merge(pd.DataFrame(adata_CRC_Tcell.var_names),annot,left_on="gene_symbol",right_on="external_gene_name",how="left")

In [16]:
annot['gene_biotype'].unique()

array([nan, 'lncRNA', 'protein_coding', 'transcribed_unitary_pseudogene',
       'snRNA', 'scaRNA', 'IG_V_pseudogene',
       'transcribed_unprocessed_pseudogene', 'polymorphic_pseudogene',
       'transcribed_processed_pseudogene', 'scRNA', 'IG_V_gene',
       'IG_C_gene', 'IG_J_gene', 'unprocessed_pseudogene', 'TR_C_gene',
       'TR_J_gene', 'TR_V_gene', 'TR_V_pseudogene', 'TR_D_gene',
       'IG_C_pseudogene', 'ribozyme', 'miRNA', 'processed_pseudogene',
       'TR_J_pseudogene', 'IG_J_pseudogene', 'IG_D_gene'], dtype=object)

In [17]:
annot= annot.drop_duplicates(["gene_symbol"])

In [18]:
annot = annot.fillna("Not_available")

In [19]:
print(adata_CRC_Tcell.n_obs, adata_CRC_Tcell.n_vars)

8788 33668


In [20]:
adata_CRC_Tcell.var['gene_biotype'] = annot.set_index("gene_symbol")['gene_biotype']


In [21]:
annot

,gene_symbol,start_position,end_position,chromosome_name,gene_biotype
0,RP11-34P13.3,Not_available,Not_available,Not_available,Not_available
1,FAM138A,34554.0,36081.0,1,lncRNA
2,OR4F5,65419.0,71585.0,1,protein_coding
3,RP11-34P13.7,Not_available,Not_available,Not_available,Not_available
4,RP11-34P13.8,Not_available,Not_available,Not_available,Not_available
...,...,...,...,...,...
36625,AC233755.2,Not_available,Not_available,Not_available,Not_available
36626,AC233755.1,Not_available,Not_available,Not_available,Not_available
36627,AC240274.1,Not_available,Not_available,Not_available,Not_available
36628,AC213203.1,Not_available,Not_available,Not_available,Not_available


In [22]:
protein_coding = annot["gene_symbol"].loc[annot['gene_biotype']=="protein_coding"].reset_index()

In [23]:
del protein_coding['index']

In [24]:
protein_gene_indicator = np.in1d(adata_CRC_Tcell.var_names, protein_coding)
adata_CRC_Tcell = adata_CRC_Tcell[:, protein_gene_indicator]

In [25]:
print(adata_CRC_Tcell.n_obs, adata_CRC_Tcell.n_vars)

8788 18258


In [26]:
adata_CRC_Tcell.raw = adata_CRC_Tcell

In [27]:
adata_CRC_Tcell.write("/mnt/lugli/spuccio/SP028_Autoimmunity/Cariplo/IBD_counts/h5files/CRC_T_cell.h5ad")

... storing 'PatientNumber' as categorical
... storing 'TumorType' as categorical
... storing 'TumorSite' as categorical
... storing 'CellType' as categorical
... storing 'gene_biotype' as categorical
... storing 'gene_biotype' as categorical
